## MySQL Flask와 연동

### 기본설정

In [14]:
!pip install flask

In [15]:
!pip install flask-restful

     ---------------------------------------- 52.8/52.8 kB 2.7 MB/s eta 0:00:00


In [16]:
!pip install flask-mysql

     ---------------------------------------- 44.8/44.8 kB ? eta 0:00:00


In [1]:
!pip install sqlalchemy

In [2]:
!pip install mysql-connector-python

     --------------------------------------- 10.8/10.8 MB 21.8 MB/s eta 0:00:00
     ------------------------------------- 527.0/527.0 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.3
    Uninstalling protobuf-4.23.3:
      Successfully uninstalled protobuf-4.23.3


In [7]:
!pip install watchdog

In [15]:
!pip install celery

     -------------------------------------- 420.4/420.4 kB 6.5 MB/s eta 0:00:00
     ------------------------------------- 198.5/198.5 kB 12.6 MB/s eta 0:00:00
     ---------------------------------------- 86.7/86.7 kB 5.1 MB/s eta 0:00:00
     ------------------------------------- 341.8/341.8 kB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 97.9/97.9 kB ? eta 0:00:00
     ------------------------------------- 385.2/385.2 kB 12.1 MB/s eta 0:00:00
     ---------------------------------------- 50.8/50.8 kB ? eta 0:00:00
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.20
    Uninstalling prompt-toolkit-3.0.20:
      Successfully uninstalled prompt-toolkit-3.0.20
  Attempting uninstall: click
    Found existing installation: click 8.0.4
    Uninstalling click-8.0.4:
      Successfully uninstalled click-8.0.4


In [17]:
!pip install redis

     -------------------------------------- 241.1/241.1 kB 7.4 MB/s eta 0:00:00


In [18]:
!pip install Flask-Mail

In [29]:
!pip install --upgrade pathlib2

In [20]:
!pip install pymysql pillow

In [73]:
from flask      import Flask, request, jsonify, current_app, render_template, redirect, url_for
from flask.json import JSONEncoder
from sqlalchemy import create_engine, text
from flaskext.mysql import MySQL
from http import HTTPStatus
import json
import pymysql
import pandas as pd
import numpy as np
import mysql.connector

In [74]:
# 서버 구동시킬 객체 
from flask import Flask

# 메일, 메세지 함수, 변수 가져오기
from flask_mail import Mail, Message

# 요청, 응답 처리할 수 있는 함수, 변수 가져오기
from flask import request, make_response, redirect, jsonify, Flask, Blueprint, render_template

# 웹 페이지 규칙
from flask_cors import CORS

import jwt
import hashlib
from datetime import datetime, timedelta
from celery import Celery
import pickle
import time

## Select

In [110]:
cctv_select = "SELECT * FROM tb_cctv1 WHERE U_ID=%s;"
sah_select = "SELECT * FROM tb_security_alarm_human WHERE U_ID=%s;"
sac_select = "SELECT * FROM tb_security_alarm_car WHERE U_ID=%s;"

In [128]:
# DB 연결 및 쿼리 작성
def db_connector(u_id):
    db = pymysql.connect(host='project-db-cgi.smhrd.com', port=3307, user='mds', passwd='mds01', db='mds', charset='utf8')
    cursor = db.cursor()
#     sql = cctv_select
    sql = "SELECT * FROM tb_security_alarm_human WHERE U_ID=%s;"
#     param = u_id
    cursor.execute(sql, [u_id])
    result = cursor.fetchall()
    db.close()
    return str(result)


In [160]:
def getList(u_id):    
    gil_db = pymysql.connect(
        host='project-db-cgi.smhrd.com', 
        port=3307, 
        user='mds', 
        passwd='mds01', 
        db='mds', 
        charset='utf8'
    )
    cursor = gil_db.cursor()
    
    #* 쓰는걸 지양하자. 파이썬 ;이 있거나 없거나 상관이 없다.
    sql = f"""
            SELECT cctv_id, car_num, car_inout_at, sac_content, sac_at, car_img_link
            FROM tb_security_alarm_car
            WHERE u_id = '{u_id}'
            """
    cursor.execute(sql)
    rows = cursor.fetchall()
#     print(rows[2][2])
    
    # price = []
    list = []
    for row in rows:
        # price.append(row[2])
        list.append(row)
    
    gil_db.close()
    return list

if __name__ == '__main__':
    arr = getList("test1")
    arr = np.array(arr)
    print(arr)

[['20231E483851250911EE894C0242AC14000B' '11바1111'
  datetime.datetime(2023, 7, 18, 12, 36, 35) '알림 내용'
  datetime.datetime(2023, 7, 18, 12, 36, 35) '일단 이미지 없음']
 ['20231E483851250911EE894C0242AC14000B' '11바 1111'
  datetime.datetime(2023, 7, 20, 11, 51, 14) '알림 내용'
  datetime.datetime(2023, 7, 20, 11, 51, 14)
  'C:UserssmhrdDesktopsdoqu.png']]


## Insert

### 행 1개 삽입

In [194]:
# 테이블에 행 1개 삽입
import mysql.connector



mydb = mysql.connector.connect(

  host="project-db-cgi.smhrd.com",
    
  port = 3307,

  user="mds",

  password="mds01",

  database="mds"

)



mycursor = mydb.cursor()

# tb_cctv1
# sql = "INSERT INTO tb_cctv1(u_id, rtsp_address, rtsp_address2, cctv_loc) VALUES(%s, %s, %s, %s)"
# val = ("test3", "rtsp://210.99.70.120:1935/live/cctv009.stream", "rtsp://210.99.70.120:1935/live/cctv015.stream", "이건 몰라유")

# tb_security_alarm_human
# sql = "INSERT INTO tb_security_alarm_human(cctv_id, sah_content, sah_at, sah_read, sah_read_at, sah_img_link, sah_total, u_id) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"
# val = ("20231E483851250911EE894C0242AC14000B", )

# tb_security_alarm_car
sql = "INSERT INTO tb_security_alarm_car(cctv_id, car_num, car_inout_at, sac_content, sac_at, sac_read, sac_read_at, sac_img_link, u_id) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
val = ("20231E483851250911EE894C0242AC14000B", "33바 3333", strftime('%Y-%m-%d %H:%M:%S'), "차량 침입!!!", strftime('%Y-%m-%d %H:%M:%S'), "N", strftime('%Y-%m-%d %H:%M:%S'), "이미지는 잠깐 나중에", "test1")

mycursor.execute(sql, val)

mydb.commit()

print(mycursor.rowcount, "record inserted.")

1 record inserted.


### 행 여러 개 삽입

In [ ]:
# 테이블에 행 여러 개 삽입
import mysql.connector


mydb = mysql.connector.connect(

  host="project-db-cgi.smhrd.com",
    
  port = 3307,

  user="mds",

  password="mds01",

  database="mds"

)



mycursor = mydb.cursor()

# tb_cctv1
sql = "INSERT INTO tb_cctv1(u_id, rtsp_address, rtsp_address2, cctv_loc) VALUES(%s, %s, %s, %s)"
# 하나하나 넣을 거 아니면 for문 추천드림★
val = [
        ("test3", "rtsp://210.99.70.120:1935/live/cctv009.stream", "rtsp://210.99.70.120:1935/live/cctv015.stream", "이건 몰라유"),
        ("test4", "rtsp://210.99.70.120:1935/live/cctv018.stream", "rtsp://210.99.70.120:1935/live/cctv020.stream", "이건 몰라유")
        ]

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "record inserted.")

In [ ]:
# 삽입된 마지막 레코드의 ID 가져오기
# mycursor.lastrowid)
# print("1 record inserted, ID:", mycursor.lastrowid) 

### 이미지 삽입하기

In [113]:
import os
from PIL import Image
import base64
from io import BytesIO
import mysql.connector
import pymysql
import time
import cv2

# DB 연결
mydb = mysql.connector.connect(
  host="project-db-cgi.smhrd.com",
    port = 3307,
    user="mds",
    password="mds01",
    database="mds"
)
mycursor = mydb.cursor()

# 폴더 내 이미지 불러오는 코드
path = "C:\\Users\\smhrd\\git\\MDS\\MDS\\src\\main\\webapp\\resources\\alarmCapture"
img_path_list = []
possible_img_extension = ['.jpg', '.jpeg', '.JPG', '.png']
for (root, dirs, files) in os.walk(path):
    if len(files) > 0:
        for file_name in files:
            if os.path.splitext(file_name)[1] in possible_img_extension:
                img_path = root + '/' + file_name
                img_path = img_path.replace('\\', '/')
                img_path_list.append(img_path)
                
print(img_path_list)
print(img_path_list[1])

# Insert
# tb_security_alarm_car
sql = "INSERT INTO tb_security_alarm_car(cctv_id, car_num, car_inout_at, sac_content, sac_at, sac_read, sac_img_link, u_id) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"
val = ("20231E483851250911EE894C0242AC14000B", "22바 2222", time.strftime('%Y-%m-%d %H:%M:%S'), "차량 침입!!!", time.strftime('%Y-%m-%d %H:%M:%S'), "N", img_path_list[0], "test1")

# # tb_security_alarm_human
# sql = "INSERT INTO tb_security_alarm_human(cctv_id, sah_content, sah_at, sah_read, sah_img_link, sah_total, u_id) VALUES(%s, %s, %s, %s, %s, %s, %s)"
# val = ("20231E483851250911EE894C0242AC14000B", '사람 침입', time.strftime('%Y-%m-%d %H:%M:%S'), 'N', img_path_list[8], 1, 'test1')

mycursor.execute(sql, val)

mydb.commit()

mydb.close()

print(mycursor.rowcount, "record inserted.")            

['C:/Users/smhrd/Desktop/image/daniel.jpg', 'C:/Users/smhrd/Desktop/image/gdragon.jpg', 'C:/Users/smhrd/Desktop/image/hsh.jpg', 'C:/Users/smhrd/Desktop/image/joebiden.jpg', 'C:/Users/smhrd/Desktop/image/karina.jpg', 'C:/Users/smhrd/Desktop/image/ktr.jpg', 'C:/Users/smhrd/Desktop/image/ldh.jpg', 'C:/Users/smhrd/Desktop/image/mds.jpg', 'C:/Users/smhrd/Desktop/image/mrr.jpg', 'C:/Users/smhrd/Desktop/image/peb.jpg', 'C:/Users/smhrd/Desktop/image/phs.jpg', 'C:/Users/smhrd/Desktop/image/psw.jpg', 'C:/Users/smhrd/Desktop/image/stu.jpg', 'C:/Users/smhrd/Desktop/image/sye.jpg', 'C:/Users/smhrd/Desktop/image/tony.jpg', 'C:/Users/smhrd/Desktop/image/yhj.jpg', 'C:/Users/smhrd/Desktop/image/ysy.jpg']
C:/Users/smhrd/Desktop/image/gdragon.jpg
1 record inserted.


## Spring 연동

### 방법 1 - 실패

In [ ]:

app = Flask(__name__)


@app.route("/main", methods=["post"])
def dataSelect():
    u_id = request.form.get("u_id")
    a = db_connector('test1')
#     print(u_id, u_pw)
#     return jsonify({"a":a, "status":HTTPStatus.OK})
    return redirect("http://localhost:8085/mds2/main.com?a="+a)




# 3. 구동시키기 
if __name__ == "__main__": # 그냥 암기혀
    app.run(host = "127.0.0.1", port = "9000")

### 방법 2 - 실패

In [103]:
from flask import Flask, jsonify, request
import requests

app = Flask(__name__)

@app.route("/", methods=["get"])
def send_data():
    a = db_connector('test1')
    data_json = jsonify(data=a)
    response = requests.get("http://localhost:8085/mds2/")
    # Response 객체에서 JSON 데이터 추출
    data_from_response = response.json()
    # 이후 필요한 작업을 수행 (예: 데이터 처리, 응답 생성 등)
    return jsonify(data_from_response) # Flask 애플리케이션에서 JSON으로 응답 생성

# 3. 구동시키기 
if __name__ == "__main__": # 그냥 암기혀
    app.run(host = "127.0.0.1", port = "9000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9000/ (Press CTRL+C to quit)
[2023-07-21 12:11:59,582] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\lib\site-packages\requests\models.py", line 971, in json
    return complexjson.loads(self.text, **kwargs)
  File "C:\Users\smhrd\anaconda3\lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "C:\Users\smhrd\anaconda3\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\smhrd\anaconda3\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 4 column 1 (char 9)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  Fi

### 방법 3 - 실패

In [95]:
app = Flask(__name__)
 
@app.route('/')
def myindex():
    return redirect("http://localhost:8085/mds2/")
 
@app.route('/list', methods =['get','post'])
def mylist():
    u_id = request.form["u_id"]
    list = getList(u_id)
    return render_template("http://localhost:8085/mds2/checkUser.com", list = list, u_id = u_id)
 
if __name__ == '__main__':
    app.run(host="127.0.0.1", port='9999')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9999/ (Press CTRL+C to quit)
[2023-07-20 15:48:47,013] ERROR in app: Exception on /list [POST]
Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\smhrd\Ap

### 방법 4 - 성공
- 비동기 ajax를 이용한 방법

In [4]:
def db_connector_sah(u_id):
    db = pymysql.connect(host='project-db-cgi.smhrd.com', port=3307, user='mds', passwd='mds01', db='mds', charset='utf8')
    cursor = db.cursor()
#     sql = cctv_select
    sql = f"SELECT * FROM tb_security_alarm_human WHERE U_ID='{u_id}';"
    cursor.execute(sql)
    # 컬럼 이름들을 가져옵니다.
    columns = [col[0] for col in cursor.description]
    
    # fetchall()로 조회된 데이터를 dict 형태로 변환하여 리스트에 추가합니다.
    result_list = []
    for row in cursor.fetchall():
        result_dict = dict(zip(columns, row))
        result_list.append(result_dict)
    db.close()
    return result_list

def db_connector_sac(u_id):
    db = pymysql.connect(host='project-db-cgi.smhrd.com', port=3307, user='mds', passwd='mds01', db='mds', charset='utf8')
    cursor = db.cursor()
#     sql = cctv_select
    sql = f"SELECT * FROM tb_security_alarm_car WHERE U_ID='{u_id}';"
    cursor.execute(sql)
    # 컬럼 이름들을 가져옵니다.
    columns = [col[0] for col in cursor.description]
    
    # fetchall()로 조회된 데이터를 dict 형태로 변환하여 리스트에 추가합니다.
    result_list = []
    for row in cursor.fetchall():
        result_dict = dict(zip(columns, row))
        result_list.append(result_dict)
    db.close()
    return result_list

app = Flask(__name__)

@app.route("/webAlarm", methods = ["POST"])
def webAlarm():
#     value = request.get_json() # sql 문법이 이상하다고 오류뜸
#     value = request.form["u_id"] # BAD REQUEST
#     value = request.json('u_id')
    value = request.form.get('u_id')
    print(value)
    lst = db_connector_sah(value)
    lst2 = db_connector_sac(value)
    print(lst)
    print(lst2)
    
    # 메일로 알림 전송
    msg = Message('Hello', sender='moondansok66@gmail.com', recipients=['guswn100059@gmail.com'])
    msg.body = '침입이 감지되었습니다.'
    mail.send(msg)
    return jsonify([lst, lst2])

    
if __name__=='__main__':
    app.run(host='127.0.0.1',port=9000)
    


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9000/ (Press CTRL+C to quit)


test1


127.0.0.1 - - [25/Jul/2023 12:45:26] "POST /webAlarm HTTP/1.1" 200 -


[{'sah_num': 100383357128409130, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_content': '알림 내용', 'sah_at': datetime.datetime(2023, 7, 18, 12, 38, 8), 'sah_read': 'n', 'sah_read_at': datetime.datetime(2023, 7, 18, 12, 38, 8), 'human_img_link': '일단 이미지 없숨', 'sah_total': 15, 'u_id': 'test1'}, {'sah_num': 100383357128409133, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_content': '알림 내용', 'sah_at': datetime.datetime(2023, 7, 21, 17, 58, 49), 'sah_read': 'n', 'sah_read_at': datetime.datetime(2023, 7, 21, 17, 58, 49), 'human_img_link': '일단 이미지 없숨', 'sah_total': 15, 'u_id': 'test1'}, {'sah_num': 100383357128409136, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_content': '사람 침입!!!', 'sah_at': datetime.datetime(2023, 7, 24, 14, 29, 20), 'sah_read': 'N', 'sah_read_at': datetime.datetime(2023, 7, 24, 14, 29, 20), 'human_img_link': '아직 사진 없숨', 'sah_total': 1, 'u_id': 'test1'}, {'sah_num': 100383357128409137, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_con

...

test1
[{'sah_num': 100383357128409130, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_content': '알림 내용', 'sah_at': datetime.datetime(2023, 7, 18, 12, 38, 8), 'sah_read': 'n', 'sah_read_at': datetime.datetime(2023, 7, 18, 12, 38, 8), 'human_img_link': '일단 이미지 없숨', 'sah_total': 15, 'u_id': 'test1'}, {'sah_num': 100383357128409133, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_content': '알림 내용', 'sah_at': datetime.datetime(2023, 7, 21, 17, 58, 49), 'sah_read': 'n', 'sah_read_at': datetime.datetime(2023, 7, 21, 17, 58, 49), 'human_img_link': '일단 이미지 없숨', 'sah_total': 15, 'u_id': 'test1'}, {'sah_num': 100383357128409136, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_content': '사람 침입!!!', 'sah_at': datetime.datetime(2023, 7, 24, 14, 29, 20), 'sah_read': 'N', 'sah_read_at': datetime.datetime(2023, 7, 24, 14, 29, 20), 'human_img_link': '아직 사진 없숨', 'sah_total': 1, 'u_id': 'test1'}, {'sah_num': 100383357128409137, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 's

127.0.0.1 - - [25/Jul/2023 12:48:32] "POST /webAlarm HTTP/1.1" 200 -


test1
[{'sah_num': 100383357128409130, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_content': '알림 내용', 'sah_at': datetime.datetime(2023, 7, 18, 12, 38, 8), 'sah_read': 'n', 'sah_read_at': datetime.datetime(2023, 7, 18, 12, 38, 8), 'human_img_link': '일단 이미지 없숨', 'sah_total': 15, 'u_id': 'test1'}, {'sah_num': 100383357128409133, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_content': '알림 내용', 'sah_at': datetime.datetime(2023, 7, 21, 17, 58, 49), 'sah_read': 'n', 'sah_read_at': datetime.datetime(2023, 7, 21, 17, 58, 49), 'human_img_link': '일단 이미지 없숨', 'sah_total': 15, 'u_id': 'test1'}, {'sah_num': 100383357128409136, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_content': '사람 침입!!!', 'sah_at': datetime.datetime(2023, 7, 24, 14, 29, 20), 'sah_read': 'N', 'sah_read_at': datetime.datetime(2023, 7, 24, 14, 29, 20), 'human_img_link': '아직 사진 없숨', 'sah_total': 1, 'u_id': 'test1'}, {'sah_num': 100383357128409137, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 's

127.0.0.1 - - [25/Jul/2023 12:50:38] "POST /webAlarm HTTP/1.1" 200 -


test1
[{'sah_num': 100383357128409130, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_content': '알림 내용', 'sah_at': datetime.datetime(2023, 7, 18, 12, 38, 8), 'sah_read': 'n', 'sah_read_at': datetime.datetime(2023, 7, 18, 12, 38, 8), 'human_img_link': '일단 이미지 없숨', 'sah_total': 15, 'u_id': 'test1'}, {'sah_num': 100383357128409133, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_content': '알림 내용', 'sah_at': datetime.datetime(2023, 7, 21, 17, 58, 49), 'sah_read': 'n', 'sah_read_at': datetime.datetime(2023, 7, 21, 17, 58, 49), 'human_img_link': '일단 이미지 없숨', 'sah_total': 15, 'u_id': 'test1'}, {'sah_num': 100383357128409136, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'sah_content': '사람 침입!!!', 'sah_at': datetime.datetime(2023, 7, 24, 14, 29, 20), 'sah_read': 'N', 'sah_read_at': datetime.datetime(2023, 7, 24, 14, 29, 20), 'human_img_link': '아직 사진 없숨', 'sah_total': 1, 'u_id': 'test1'}, {'sah_num': 100383357128409137, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 's

### 종합
- insert 후 select한 데이터들을 spring으로 연동

In [201]:
# 테이블에 행 1개 삽입
import mysql.connector

mydb = mysql.connector.connect(

  host="project-db-cgi.smhrd.com",
    
  port = 3307,

  user="mds",

  password="mds01",

  database="mds"

)

mycursor = mydb.cursor()

# tb_cctv1
# sql = "INSERT INTO tb_cctv1(u_id, rtsp_address, rtsp_address2, cctv_loc) VALUES(%s, %s, %s, %s)"
# val = ("test3", "rtsp://210.99.70.120:1935/live/cctv009.stream", "rtsp://210.99.70.120:1935/live/cctv015.stream", "이건 몰라유")

# tb_security_alarm_human
sql = "INSERT INTO tb_security_alarm_human(cctv_id, sah_content, sah_at, sah_read, sah_read_at, human_img_link, sah_total, u_id) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"
val = ("20231E483851250911EE894C0242AC14000B", "사람 침입!!!", time.strftime('%Y-%m-%d %H:%M:%S'), "N", time.strftime('%Y-%m-%d %H:%M:%S'), "아직 사진 없숨", 1, 'test1')

# tb_security_alarm_car
# sql = "INSERT INTO tb_security_alarm_car(cctv_id, car_num, car_inout_at, sac_content, sac_at, sac_read, sac_read_at, car_img_link, u_id) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
# val = ("20231E483851250911EE894C0242AC14000B", "33바 3333", time.strftime('%Y-%m-%d %H:%M:%S'), "차량 침입!!!", time.strftime('%Y-%m-%d %H:%M:%S'), "N", time.strftime('%Y-%m-%d %H:%M:%S'), "이미지는 잠깐 나중에", "test1")

mycursor.execute(sql, val)

mydb.commit()

print(mycursor.rowcount, "record inserted.")

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Select 쿼리문
def db_connector_sah(u_id):
    db = pymysql.connect(host='project-db-cgi.smhrd.com', port=3307, user='mds', passwd='mds01', db='mds', charset='utf8')
    cursor = db.cursor()
#     sql = cctv_select
    sql = f"SELECT * FROM tb_security_alarm_human WHERE U_ID='{u_id}';"
    cursor.execute(sql)
    # 컬럼 이름들을 가져옵니다.
    columns = [col[0] for col in cursor.description]
    
    # fetchall()로 조회된 데이터를 dict 형태로 변환하여 리스트에 추가합니다.
    result_list = []
    for row in cursor.fetchall():
        result_dict = dict(zip(columns, row))
        result_list.append(result_dict)
    db.close()
    return result_list

def db_connector_sac(u_id):
    db = pymysql.connect(host='project-db-cgi.smhrd.com', port=3307, user='mds', passwd='mds01', db='mds', charset='utf8')
    cursor = db.cursor()
#     sql = cctv_select
    sql = f"SELECT * FROM tb_security_alarm_car WHERE U_ID='{u_id}';"
    cursor.execute(sql)
    # 컬럼 이름들을 가져옵니다.
    columns = [col[0] for col in cursor.description]
    
    # fetchall()로 조회된 데이터를 dict 형태로 변환하여 리스트에 추가합니다.
    result_list = []
    for row in cursor.fetchall():
        result_dict = dict(zip(columns, row))
        result_list.append(result_dict)
    db.close()
    return result_list


app = Flask(__name__)


@app.route("/webAlarm", methods = ["POST"])
def webAlarm():
#     value = request.get_json() # sql 문법이 이상하다고 오류뜸
#     value = request.form["u_id"] # BAD REQUEST
#     value = request.json('u_id')
    value = request.form.get('u_id')
    print(value)
    lst = db_connector_sah(value)
    lst2 = db_connector_sac(value)
    print(lst)
    print(lst2)
    return jsonify([lst, lst2])

    
if __name__=='__main__':
    app.run(host='127.0.0.1',port=9000)


1 record inserted.
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Jul/2023 14:32:54] "POST /webAlarm HTTP/1.1" 200 -


test1
[{'sac_num': 100383357128409129, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'car_num': '11바1111', 'car_inout_at': datetime.datetime(2023, 7, 18, 12, 36, 35), 'sac_content': '알림 내용', 'sac_at': datetime.datetime(2023, 7, 18, 12, 36, 35), 'sac_read': 'y', 'sac_read_at': datetime.datetime(2023, 7, 18, 12, 36, 35), 'car_img_link': '일단 이미지 없음', 'u_id': 'test1'}, {'sac_num': 100383357128409132, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'car_num': '11바 1111', 'car_inout_at': datetime.datetime(2023, 7, 20, 11, 51, 14), 'sac_content': '알림 내용', 'sac_at': datetime.datetime(2023, 7, 20, 11, 51, 14), 'sac_read': 'N', 'sac_read_at': datetime.datetime(2023, 7, 20, 11, 51, 14), 'car_img_link': 'C:UserssmhrdDesktopsdoqu.png', 'u_id': 'test1'}, {'sac_num': 100383357128409134, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'car_num': '22바 2222', 'car_inout_at': datetime.datetime(2023, 7, 24, 13, 9, 42), 'sac_content': '차량 침입!!!', 'sac_at': datetime.datetime(2023, 7, 24, 13, 9, 4

## 이메일 알림 전송

In [25]:
celery = Celery(app.name, broker='redis://localhost:6379/0')

In [26]:
app.config['SECRET_KEY'] = 'not_a_secret'

In [28]:
app = Flask(__name__)

mail = Mail(app)

app.config['MAIL_SERVER'] = 'smtp.gmail.com'
app.config['MAIL_PORT'] = 465
app.config['MAIL_USERNAME'] = 'moondansok66@gmail.com'
app.config['MAIL_PASSWORD'] = 'unqojkesjiynxogd'
app.config['MAIL_USE_TLS'] = False
app.config['MAIL_USE_SSL'] = True
mail = Mail(app)

@app.route('/')
def email():
    msg = Message('Hello', sender='moondansok66@gmail.com', recipients=['guswn100059@gmail.com'])
    msg.body = '침입이 감지되었습니다.'
    mail.send(msg)
    return 'Sent'

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

## 웹 알림 + 메일 종합

In [10]:
import os
from PIL import Image
import base64
from io import BytesIO
import mysql.connector
import pymysql
import time
import cv2

# DB 연결
mydb = mysql.connector.connect(
  host="project-db-cgi.smhrd.com",
    port = 3307,
    user="mds",
    password="mds01",
    database="mds"
)
mycursor = mydb.cursor()

# 폴더 내 이미지 불러오는 코드
# tb_security_alarm_human
path_h = "C:\\Users\\smhrd\\git\\MDS\\MDS\\src\\main\\webapp\\resources\\alarmCapture_h"
img_path_list_h = []
possible_img_extension = ['.jpg', '.jpeg', '.JPG', '.png']
for (root, dirs, files) in os.walk(path_h):
    if len(files) > 0:
        for file_name in files:
            if os.path.splitext(file_name)[1] in possible_img_extension:
                img_path = root + '/' + file_name
                img_path = img_path.replace('\\', '/')
                img_path_list_h.append(img_path)

# 폴더 내 이미지 불러오는 코드
# tb_security_alarm_car
path_c = "C:\\Users\\smhrd\\git\\MDS\\MDS\\src\\main\\webapp\\resources\\alarmCapture_c"
img_path_list_c = []
possible_img_extension = ['.jpg', '.jpeg', '.JPG', '.png']
for (root, dirs, files) in os.walk(path_c):
    if len(files) > 0:
        for file_name in files:
            if os.path.splitext(file_name)[1] in possible_img_extension:
                img_path = root + '/' + file_name
                img_path = img_path.replace('\\', '/')
                img_path_list_c.append(img_path)
# print(img_path_list)
# print(img_path_list[1])

# Insert
# tb_security_alarm_car
# sql = "INSERT INTO tb_security_alarm_car(cctv_id, car_num, sac_content, sac_read, sac_img_link, u_id, sac_in_count, sac_out_count, sac_in_at, sac_out_at) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
# val = ("20231E483851250911EE894C0242AC14000B", "22바 2222", "차량 침입이 감지되었습니다.", "N", img_path_list_c[0], "test1", 1, 1, time.strftime('%Y-%m-%d %H:%M:%S'), time.strftime('%Y-%m-%d %H:%M:%S'))

# # tb_security_alarm_human
sql = "INSERT INTO tb_security_alarm_human(cctv_id, sah_content, sah_read, sah_img_link, u_id, sah_in_count, sah_out_count, sah_in_at, sah_out_at) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
val = ("20231E483851250911EE894C0242AC14000B", '사람 침입이 감지되었습니다.', 'N', img_path_list_h[5], 'test1', 1, 2, time.strftime('%Y-%m-%d %H:%M:%S'), time.strftime('%Y-%m-%d %H:%M:%S'))

mycursor.execute(sql, val)

mydb.commit()

mydb.close()

print(mycursor.rowcount, "record inserted.")            

1 record inserted.


In [19]:
# Select 쿼리문
def db_connector_sah(u_id):
    db = pymysql.connect(host='project-db-cgi.smhrd.com', port=3307, user='mds', passwd='mds01', db='mds', charset='utf8')
    cursor = db.cursor()
#     sql = cctv_select
    sql = f"SELECT * FROM tb_security_alarm_human WHERE U_ID='{u_id}' AND SAH_READ='N';"
    cursor.execute(sql)
    # 컬럼 이름들을 가져옵니다.
    columns = [col[0] for col in cursor.description]
    
    # fetchall()로 조회된 데이터를 dict 형태로 변환하여 리스트에 추가합니다.
    result_list = []
    for row in cursor.fetchall():
        result_dict = dict(zip(columns, row))
        result_list.append(result_dict)
    db.close()
    return result_list

def db_connector_sac(u_id):
    db = pymysql.connect(host='project-db-cgi.smhrd.com', port=3307, user='mds', passwd='mds01', db='mds', charset='utf8')
    cursor = db.cursor()
#     sql = cctv_select
    sql = f"SELECT * FROM tb_security_alarm_car WHERE U_ID='{u_id}' AND SAC_READ='N';"
    cursor.execute(sql)
    # 컬럼 이름들을 가져옵니다.
    columns = [col[0] for col in cursor.description]
    
    # fetchall()로 조회된 데이터를 dict 형태로 변환하여 리스트에 추가합니다.
    result_list = []
    for row in cursor.fetchall():
        result_dict = dict(zip(columns, row))
        result_list.append(result_dict)
    db.close()
    return result_list

# 이메일 
def db_connector_email(u_id):
    db = pymysql.connect(host='project-db-cgi.smhrd.com', port=3307, user='mds', passwd='mds01', db='mds', charset='utf8')
    cursor = db.cursor()
#     sql = cctv_select
    sql = f"SELECT u_email FROM tb_user WHERE U_ID='{u_id}';"
    cursor.execute(sql)
    # fetchall()로 조회된 데이터를 튜플에서 문자열로 변환하여 리스트에 추가합니다.
    result_list = [row[0] for row in cursor.fetchall()]
    db.close()
    return result_list
    

app = Flask(__name__)

mail = Mail(app)

app.config['MAIL_SERVER'] = 'smtp.gmail.com'
app.config['MAIL_PORT'] = 465
app.config['MAIL_USERNAME'] = 'moondansok66@gmail.com'
app.config['MAIL_PASSWORD'] = 'unqojkesjiynxogd'
app.config['MAIL_USE_TLS'] = False
app.config['MAIL_USE_SSL'] = True
mail = Mail(app)

@app.route("/webAlarm", methods = ["POST"])
def webAlarm():
    # 웹으로 알림 전송
    value = request.form.get('u_id')
    print(value)
    lst = db_connector_sah(value)
    lst2 = db_connector_sac(value)
    print(lst)
    print(lst2)
    
    # BigInt 데이터타입 => str로 변환
    for i in range(len(lst)):
        lst[i]['sah_num'] = str(lst[i]['sah_num'])
        lst[i]['sah_in_at'] = str(lst[i]['sah_in_at'])
        lst[i]['sah_out_at'] = str(lst[i]['sah_out_at'])
        
    for i in range(len(lst2)):
        lst2[i]['sac_num'] = str(lst2[i]['sac_num'])
        lst2[i]['sac_in_at'] = str(lst2[i]['sac_in_at'])
        lst2[i]['sac_out_at'] = str(lst2[i]['sac_out_at'])
    
    
    # 메일 알림 전송
    email = db_connector_email(value)
    print(email)
    msg = Message('🚨침입감지🚨', sender='moondansok66@gmail.com', recipients=email)
    msg.body = '침입이 감지되었습니다.'
    mail.send(msg)
    
    return jsonify([lst, lst2])

# for i in range(len(db_connector_sah('test1'))):
#     print(db_connector_sah('test1')[i]['sah_num'])
# # print(len(db_connector_sac('test1')))
# for i in range(len(db_connector_sac('test1'))):
#     db_connector_sac('test1')[i]['sac_num'] = str(db_connector_sac('test1')[i]['sac_num'])
#     print(db_connector_sac('test1')[i]['sac_num'])

    
if __name__=='__main__':
    app.run(host='127.0.0.1',port=9000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9000/ (Press CTRL+C to quit)


In [51]:
def mainpage(u_id):
    db = pymysql.connect(host='project-db-cgi.smhrd.com', port=3307, user='mds', passwd='mds01', db='mds', charset='utf8')
    cursor = db.cursor()
    sql = f"select A.*, B.* from tb_security_alarm_car as A join tb_security_alarm_human as B on A.u_id = B.u_id where A.u_id = '{u_id}' order by A.sac_in_at desc limit 4"
    cursor.execute(sql)
    # 컬럼 이름들을 가져옵니다.
    columns = [col[0] for col in cursor.description]
    
    # fetchall()로 조회된 데이터를 dict 형태로 변환하여 리스트에 추가합니다.
    result_list = []
    for row in cursor.fetchall():
        result_dict = dict(zip(columns, row))
        result_list.append(result_dict)
    db.close()
    return result_list

mainpage('test1')[1]['sah_num']

100414190262419666

In [71]:
def mainpage(u_id):
    db = pymysql.connect(host='project-db-cgi.smhrd.com', port=3307, user='mds', passwd='mds01', db='mds', charset='utf8')
    cursor = db.cursor()
    sql = f"select A.*, B.* from tb_security_alarm_car as A join tb_security_alarm_human as B on A.u_id = B.u_id where A.u_id = '{u_id}' order by A.sac_in_at desc limit 4"
    cursor.execute(sql)
    # 컬럼 이름들을 가져옵니다.
    columns = [col[0] for col in cursor.description]
    
    # fetchall()로 조회된 데이터를 dict 형태로 변환하여 리스트에 추가합니다.
    result_list = []
    for row in cursor.fetchall():
        result_dict = dict(zip(columns, row))
        result_list.append(result_dict)
    db.close()
    return result_list

mainpage('test1')[0]['sah_in_at']

test = mainpage('test1')[0]['sah_in_at'].strftime("%Y년 %m월 %d일 %H시 %M분 %S초")
print(test)
print(type(test))

lst = mainpage('test1')
lst
lst[0]['sah_in_at'].strftime("%Y년 %m월 %d일 %H시 %M분 %S초")

2023년 08월 02일 16시 22분 19초
<class 'str'>


'2023년 08월 02일 16시 22분 19초'

In [78]:
def mainpage(u_id):
    db = pymysql.connect(host='project-db-cgi.smhrd.com', port=3307, user='mds', passwd='mds01', db='mds', charset='utf8')
    cursor = db.cursor()
    sql = f"select A.*, B.* from tb_security_alarm_car as A join tb_security_alarm_human as B on A.u_id = B.u_id where A.u_id = '{u_id}' order by A.sac_in_at desc limit 4"
    cursor.execute(sql)
    # 컬럼 이름들을 가져옵니다.
    columns = [col[0] for col in cursor.description]
    
    # fetchall()로 조회된 데이터를 dict 형태로 변환하여 리스트에 추가합니다.
    result_list = []
    for row in cursor.fetchall():
        result_dict = dict(zip(columns, row))
        result_list.append(result_dict)
    db.close()
    return result_list

app = Flask(__name__)
@app.route("/connectMain", methods = ["POST"])
def test():
    u_id = request.form.get('login_id')
    print(u_id)
    lst = mainpage(u_id)
    print(lst)
    # BigInt 데이터타입 => str로 변환
    for i in range(len(lst)):
        lst[i]['sah_in_at'] = str(lst[i]['sah_in_at'])
        lst[i]['sah_out_at'] = str(lst[i]['sah_out_at'])
        lst[i]['sac_in_at'] = str(lst[i]['sac_in_at'])
        lst[i]['sac_out_at'] = str(lst[i]['sac_out_at'])
        
    return jsonify([lst])

if __name__=='__main__':
    app.run(host='127.0.0.1',port=9000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Aug/2023 11:01:23] "POST /connectMain HTTP/1.1" 200 -


test1
[{'sac_num': 100414190262419676, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'car_num': '22바 2222', 'sac_content': '차량 침입이 감지되었습니다.', 'sac_read': 'Y', 'sac_read_at': datetime.datetime(2023, 8, 4, 21, 13, 23), 'sac_img_link': 'C:/Users/smhrd/git/MDS/MDS/src/main/webapp/resources/alarmCapture_c/daniel.jpg', 'u_id': 'test1', 'sac_video_link': None, 'sac_in_count': 1, 'sac_out_count': 1, 'sac_in_at': datetime.datetime(2023, 8, 4, 19, 16, 2), 'sac_out_at': datetime.datetime(2023, 8, 4, 19, 16, 2), 'sah_num': 100414190262419657, 'sah_content': '사람 침입이 감지되었습니다.', 'sah_read': 'Y', 'sah_read_at': datetime.datetime(2023, 8, 2, 16, 23, 29), 'sah_img_link': 'C:/Users/smhrd/git/MDS/MDS/src/main/webapp/resources/alarmCapture_h/hsh.jpg', 'sah_video_link': None, 'sah_out_count': 2, 'sah_in_count': 1, 'sah_in_at': datetime.datetime(2023, 8, 2, 16, 22, 19), 'sah_out_at': datetime.datetime(2023, 8, 2, 16, 22, 19)}, {'sac_num': 100414190262419676, 'cctv_id': '20231E483851250911EE894C0242AC140

127.0.0.1 - - [05/Aug/2023 11:05:31] "POST /connectMain HTTP/1.1" 200 -


test1
[{'sac_num': 100414190262419676, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'car_num': '22바 2222', 'sac_content': '차량 침입이 감지되었습니다.', 'sac_read': 'Y', 'sac_read_at': datetime.datetime(2023, 8, 4, 21, 13, 23), 'sac_img_link': 'C:/Users/smhrd/git/MDS/MDS/src/main/webapp/resources/alarmCapture_c/daniel.jpg', 'u_id': 'test1', 'sac_video_link': None, 'sac_in_count': 1, 'sac_out_count': 1, 'sac_in_at': datetime.datetime(2023, 8, 4, 19, 16, 2), 'sac_out_at': datetime.datetime(2023, 8, 4, 19, 16, 2), 'sah_num': 100414190262419657, 'sah_content': '사람 침입이 감지되었습니다.', 'sah_read': 'Y', 'sah_read_at': datetime.datetime(2023, 8, 2, 16, 23, 29), 'sah_img_link': 'C:/Users/smhrd/git/MDS/MDS/src/main/webapp/resources/alarmCapture_h/hsh.jpg', 'sah_video_link': None, 'sah_out_count': 2, 'sah_in_count': 1, 'sah_in_at': datetime.datetime(2023, 8, 2, 16, 22, 19), 'sah_out_at': datetime.datetime(2023, 8, 2, 16, 22, 19)}, {'sac_num': 100414190262419676, 'cctv_id': '20231E483851250911EE894C0242AC140

...

[{'sac_num': 100414190262419832, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'car_num': '64너3985', 'sac_content': '차량 침입이 감지되었습니다.', 'sac_read': 'N', 'sac_read_at': None, 'sac_img_link': 'C:/Users/smhrd/git/MDS/MDS/src/main/webapp/resources/alarmCapture_c/car3.png', 'u_id': 'test3', 'sac_video_link': None, 'sac_in_count': 1, 'sac_out_count': 2, 'sac_in_at': datetime.datetime(2023, 8, 7, 10, 18, 58), 'sac_out_at': datetime.datetime(2023, 8, 7, 10, 18, 58), 'sac_img_link_two': 'C:/Users/smhrd/git/MDS/MDS/src/main/webapp/resources/alarmCapture_c1/car5.png', 'sah_num': 100414190262419835, 'sah_content': '사람 침입이 감지되었습니다.', 'sah_read': 'N', 'sah_read_at': None, 'sah_img_link': 'C', 'sah_video_link': None, 'sah_out_count': 1, 'sah_in_count': 0, 'sah_in_at': datetime.datetime(2023, 8, 7, 10, 18, 58), 'sah_out_at': datetime.datetime(2023, 8, 7, 10, 18, 58), 'sah_img_link_two': 'C'}, {'sac_num': 100414190262419833, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'car_num': '27허7862', '

127.0.0.1 - - [07/Aug/2023 11:02:53] "POST /connectMain HTTP/1.1" 200 -


test3
[{'sac_num': 100414190262419832, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'car_num': '64너3985', 'sac_content': '차량 침입이 감지되었습니다.', 'sac_read': 'N', 'sac_read_at': None, 'sac_img_link': 'C:/Users/smhrd/git/MDS/MDS/src/main/webapp/resources/alarmCapture_c/car3.png', 'u_id': 'test3', 'sac_video_link': None, 'sac_in_count': 1, 'sac_out_count': 2, 'sac_in_at': datetime.datetime(2023, 8, 7, 10, 18, 58), 'sac_out_at': datetime.datetime(2023, 8, 7, 10, 18, 58), 'sac_img_link_two': 'C:/Users/smhrd/git/MDS/MDS/src/main/webapp/resources/alarmCapture_c1/car5.png', 'sah_num': 100414190262419835, 'sah_content': '사람 침입이 감지되었습니다.', 'sah_read': 'N', 'sah_read_at': None, 'sah_img_link': 'C', 'sah_video_link': None, 'sah_out_count': 1, 'sah_in_count': 0, 'sah_in_at': datetime.datetime(2023, 8, 7, 10, 18, 58), 'sah_out_at': datetime.datetime(2023, 8, 7, 10, 18, 58), 'sah_img_link_two': 'C'}, {'sac_num': 100414190262419833, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'car_num': '27허78

127.0.0.1 - - [07/Aug/2023 13:57:26] "POST /connectMain HTTP/1.1" 200 -


test3
[{'sac_num': 100414190262419845, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'car_num': '27허7862', 'sac_content': '차량 침입이 감지되었습니다.', 'sac_read': 'N', 'sac_read_at': None, 'sac_img_link': 'C:/Users/smhrd/git/MDS/MDS/src/main/webapp/resources/alarmCapture_c1/car6.png', 'u_id': 'test3', 'sac_video_link': None, 'sac_in_count': 1, 'sac_out_count': 6, 'sac_in_at': datetime.datetime(2023, 8, 7, 12, 15, 9), 'sac_out_at': datetime.datetime(2023, 8, 7, 12, 15, 9), 'sac_img_link_two': 'C:/Users/smhrd/git/MDS/MDS/src/main/webapp/resources/alarmCapture_c1/car5.png', 'sah_num': 100414190262419839, 'sah_content': '사람 침입이 감지되었습니다.', 'sah_read': 'N', 'sah_read_at': None, 'sah_img_link': 'C', 'sah_video_link': None, 'sah_out_count': 1, 'sah_in_count': 0, 'sah_in_at': datetime.datetime(2023, 8, 7, 11, 56, 29), 'sah_out_at': datetime.datetime(2023, 8, 7, 11, 56, 29), 'sah_img_link_two': 'C'}, {'sac_num': 100414190262419845, 'cctv_id': '20231E483851250911EE894C0242AC14000B', 'car_num': '27허786